In [1]:
using Pkg
Pkg.activate("/Users/jiyong/development/Projects/nImageData.jl")
using Images, Interpolations, TestImages, ImageTransformations, CoordinateTransformations, Rotations, FFTW
using LinearAlgebra
using CairoMakie
import Base: promote_rule


  Activating project at `~/development/Projects/nImageData.jl`


In [ ]:

"""
    ImageData{T}

Uniform spacing grid data as images.
"""
struct ImageData{T} <: AbstractArray{T, 2}
    mat::Matrix{T}
    
    function ImageData(mat::Matrix{T}) where T<:Real
        return new{T}(mat)
    end
end

Matrix{T}(idt::ImageData) where T= Matrix{T}(idt.mat)

Base.eltype(idt::ImageData) = Base.eltype(idt.mat)
Base.length(idt::ImageData) = Base.length(idt.mat)
Base.iterate(idt::ImageData) = Base.iterate(idt.mat)
Base.iterate(idt::ImageData, state...) = Base.iterate(idt.mat, state...)
Base.isdone(idt::ImageData; state...) = Base.isdone(idt.mat; state...)
Base.size(idt::ImageData{T}, x2...) where T = Base.size(idt.mat, x2...)


function Base.show(io::IO, idt::ImageData{T}) where T
    h, w = size(idt.mat)
    println(io, "$h","x","$w ImageData{$T}")
end

Base.getindex(idt::ImageData{T}, idx...) where T= Base.getindex(idt.mat, idx...)
Base.getindex(idt::ImageData{T}, inds::Vararg{Int,N}) where {T,N} = idt.mat[inds...]

Base.setindex!(idt::ImageData{T}, idx...) where T = Base.setindex!(idt.mat, idx...)
Base.setindex!(idt::ImageData{T}, val, inds::Vararg{Int, 2}) where T = A.data[inds...] = val

Base.showarg(io::IO, idt::ImageData, toplevel) = print(io, typeof(idt))

Base.firstindex(idt::ImageData{T}) where T = Base.firstindex(idt.mat)
Base.lastindex(idt::ImageData{T}) where T = Base.lastindex(idt.mat)

Base.BroadcastStyle(::Type{ImageData}) = ImageData()

Base.axes(idt::ImageData{T}) where T = Base.axes(idt.mat)

# Start of broadcast part to maintain shape after operation
Base.BroadcastStyle(::Type{<:ImageData}) = Broadcast.ArrayStyle{ImageData}()

function Base.similar(bc::Broadcast.Broadcasted{Broadcast.ArrayStyle{ImageData}}, ::Type{ElType}) where ElType
    # Scan the inputs for the ImageData:
    A = find_aac(bc)
    # Use the char field of A to create the output
    ImageData(similar(Array{ElType}, axes(bc)))
end

"`A = find_aac(As)` returns the first ArrayAndChar among the arguments."
find_aac(bc::Base.Broadcast.Broadcasted) = find_aac(bc.args)
find_aac(args::Tuple) = find_aac(find_aac(args[1]), Base.tail(args))
find_aac(x) = x
find_aac(::Tuple{}) = nothing
find_aac(a::ImageData, rest) = a
find_aac(::Any, rest) = find_aac(rest)

find_aac (generic function with 6 methods)

In [ ]:

interpolation_methods = Dict(:nearest => BSpline(Constant()), :bilinear => BSpline(Linear()), :bicubic => BSpline(Cubic(Line(OnGrid()))))


function rotate(imd::ImageData{T}, θ; rotation_center = nothing, method = :bilinear, fillvalue = 0) where T
    h, w = size(imd)
    if rotation_center == nothing 
        rotation_center = center(imd.mat)
    end
    trfm = recenter(RotMatrix(θ), rotation_center)
    return ImageData(warp(imd.mat, trfm, method = interpolation_methods[method], fillvalue = fillvalue)[1:h, 1:w])
end




rotate (generic function with 1 method)

In [4]:
A = reshape(collect(1.0:12), (3, 4))
M = ImageData(A)

3×4 ImageData{Float64}:
 1.0  4.0  7.0  10.0
 2.0  5.0  8.0  11.0
 3.0  6.0  9.0  12.0

In [5]:
sin.(M)

3×4 ImageData{Float64}:
 0.841471  -0.756802  0.656987  -0.544021
 0.909297  -0.958924  0.989358  -0.99999
 0.14112   -0.279415  0.412118  -0.536573

In [ ]:
X = imresize(testimage("cameraman"), (256,256))

In [ ]:
idt1 = ImageData(Float32.(X))

In [ ]:
exp.(idt1)

In [ ]:
similar(idt1, Float64)


In [ ]:
s=rotate(idt1, π/6, method = :bilinear)

In [ ]:
heatmap(s.mat)

In [ ]:
K = (size(X) .- (1, 1))./(sz .- (1, 1))
b = (1, 1) .- K
ϕ(p) = @. K*p + b


In [ ]:
for p in CartesianIndices(Y)
    q = ϕ(p.I)
    Y[p] = τ(q)
end

mosaic(X, Y; nrow=1)

In [ ]:
t = recenter(RotMatrix(π/4), center(X))

In [ ]:
center(X)

In [ ]:
extrema(X)

In [ ]:
extrema(idt1)

In [ ]:
A=rand(10, 10)
convert.(Float32, A)

In [ ]:
ca=CartesianIndex(1,2)

In [ ]:
typeof([1 2;3 4])

In [ ]:
Base.convert